# Introduction

In this mission, we'll explore how queries are executed in SQLite. After exploring this at a high level, we will explore how to create and use indexes for better performance. As our data gets larger and our queries more complex, it's important to be able to tweak the queries we write and optimize a database's schema to ensure that we're getting results back quickly.

We'll work with data from the [CIA World Factbook](https://www.cia.gov/library/publications/the-world-factbook/), a compendium of statistics about all of the countries on Earth. We'll be working with the `facts` table in the database. Each row in `facts` represents a single country, and contains several columns, including:

 - `name` -- the name of the country.
 - `area` -- the total land and sea area of the country.
 - `population` -- the population of the country.
 - `birth_rate` -- the birth rate of the country.
 - `created_at` -- the date the record was created.
 - `updated_at` -- the date the record was updated.

## The Data

 - Write a query that returns the schema of the facts table and assign the resulting list of tuples to schema.
 - Use a for loop and a print statement to display each tuple in schema on a separate line.

In [1]:
import sqlite3
conn = sqlite3.connect("factbook.db")
schema = conn.execute("pragma table_info(facts);").fetchall()
for s in schema:
    print(s)

(0, 'id', 'INTEGER', 1, None, 1)
(1, 'code', 'varchar(255)', 1, None, 0)
(2, 'name', 'varchar(255)', 1, None, 0)
(3, 'area', 'integer', 0, None, 0)
(4, 'area_land', 'integer', 0, None, 0)
(5, 'area_water', 'integer', 0, None, 0)
(6, 'population', 'integer', 0, None, 0)
(7, 'population_growth', 'float', 0, None, 0)
(8, 'birth_rate', 'float', 0, None, 0)
(9, 'death_rate', 'float', 0, None, 0)
(10, 'migration_rate', 'float', 0, None, 0)


 - Return the query plan for the query that returns all columns and rows where area exceeds 40000. Assign the results to query_plan_one.
 - Return the query plan for the query that returns only the area column for all rows where area exceeds 40000. Assign the results to query_plan_two.
 - Return the query plan for the query that returns the row for the country Czech Republic. Assign the results to query_plan_three.
 - Use the print function to display each query plan.

In [2]:
query_plan_one = conn.execute("explain query plan select * from facts where area > 40000;").fetchall()
query_plan_two = conn.execute("explain query plan select area from facts where area > 40000;").fetchall()
query_plan_three = conn.execute("explain query plan select * from facts where name = 'Czech Republic';").fetchall()

print(query_plan_one)
print(query_plan_two)
print(query_plan_three)

[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SCAN TABLE facts')]
[(2, 0, 0, 'SCAN TABLE facts')]


 - Return the query plan for the query that selects the row at id value 20 from the facts table.
 - Assign the query plan to query_plan_four and display the query plan using the print function.

In [3]:
query_plan_four = conn.execute("explain query plan select * from facts where id = 20;").fetchall()
print(query_plan_four)

[(2, 0, 0, 'SEARCH TABLE facts USING INTEGER PRIMARY KEY (rowid=?)')]


 - Return the query plan for the query that returns all values in the rows in facts where the population exceeds 10000. Assign the resulting query plan to query_plan_six and display using the print function.
 
 - Create an index for the population column in the facts table named pop_idx.
 
 - Return the query plan for the query that returns all values in the rows in facts where the population exceeds 10000. Assign the resulting query plan to query_plan_seven and display using the print function.



In [4]:
query_plan_six = conn.execute("explain query plan select * from facts where population > 10000 ;").fetchall()
print(query_plan_six)
conn.execute("create index if not exists pop_idx on facts(population)")
query_plan_seven = conn.execute("explain query plan select * from facts where population > 10000 ;").fetchall()
print(query_plan_seven)

[(2, 0, 0, 'SCAN TABLE facts')]
[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


## Multi-column db indexing

In [5]:
# conn = sqlite3.connect("factbook.db")
query_plan_one = conn.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_one)

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_idx (population>?)')]


- Create an index called pop_idx for the population column in the facts table.


 - Create an index called pop_growth_idx for the population_growth column in the facts table.

 - Return the query plan for a query that returns all rows where population is greater than 1000000 and where population_growth is less than 0.05. We're interested in all of the columns in the rows.

- Assign the query plan to query_plan_two and display it using the print function.

In [6]:
conn.execute("create index if not exists pop_idx on facts(population);").fetchall()
conn.execute("create index if not exists pop_growth_idx on facts(population_growth);").fetchall()
query_plan_two = conn.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_two)

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_growth_idx (population_growth<?)')]


 - Create a multi-column index for population and population_growth named pop_pop_growth_idx with population as the primary key.
 - Return the query plan for a query that returns all rows where population is greater than 1000000 and where population_growth is less than 0.05. We're interested in all of the columns in the rows.
 - Assign the returned query plan to query_plan_three and use the print function to display it.

In [7]:
conn.execute("create index if not exists pop_pop_growth_idx on facts(population, population_growth);")
query_plan_three = conn.execute("explain query plan select * from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_three)

[(3, 0, 0, 'SEARCH TABLE facts USING INDEX pop_pop_growth_idx (population>?)')]


This time, SQLite used the multi-column index pop_pop_growth_idx that we created instead of either pop_growth_idx or pop_idx. SQLite only needed to access the facts table to return the rest of the column values for the rows that met the WHERE criteria. This is only because the pop_pop_growth_idx doesn't contain the other values (besides population and population_growth already).

What if we restricted the columns in the SELECT that we want returned to just population and population_growth? In this case, SQLite will not need to interact with the facts table since the pop_pop_growth_idx can service the query. When an index contains all of the information necessary to answer a query, it's called a covering index. Since the index covers for the actual table and can return the requested results to the query, SQLite doesn't need to query the actual table. For many queries, especially as your data gets larger, this can be much more efficient.

Let's write a query that uses the index we created as a covering index and return its query plan.
.

In [8]:
conn.execute("create index if not exists pop_pop_growth_idx on facts(population, population_growth);")
query_plan_four = conn.execute("explain query plan select population, population_growth from facts where population > 1000000 and population_growth < 0.05;").fetchall()
print(query_plan_four)

[(2, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_pop_growth_idx (population>?)')]


Covering indexes doesn't apply just to multi-column indexes. If a query we write only touches a column in the database that we have a single-column index for, SQLite will use only the index to service the query. Let's test this by writing a query that can take advantage of just the index, pop_idx, for the population column.

In [9]:
conn.execute("create index if not exists pop_pop_growth_idx on facts(population, population_growth);")
query_plan_five = conn.execute("explain query plan select population from facts where population > 1000000;").fetchall()

print(query_plan_five)

[(2, 0, 0, 'SEARCH TABLE facts USING COVERING INDEX pop_idx (population>?)')]
